### Gerekli kütüphaneleri yüklemek ve fonksiyonları tanımlamak için başlamadan önce bu hücreyi çalıştırın.

In [112]:
import numpy as np
import pandas as pd
from sklearn import model_selection as ms
from sklearn.metrics import confusion_matrix

### Veri setini Pandas ile açın, '?' olan hücreleri NaN olarak tanımlamanız gerekiyor. (read_csv metodunda mevcut)

In [97]:
df = pd.read_csv("chronic_kidney_disease.csv",na_values="?")
df

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,51.0,7200.0,5.9,no,no,no,good,no,no,notckd


### Veri setindeki nümerik ve kategorik kolonları liste olarak yazın. Kategorik, nümerik ve hedef DataFramelerini oluşturun.

In [98]:
col_num = ["age","bp","sg","al","su", 'bgr', 'bu','sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc',] #liste
col_cat = ["rbc","pc","pcc","ba",'htn', 'dm', 'cad','appet', 'pe', 'ane'] #liste
col_tar = 'class' #string

df_num = df[col_num]
df_cat = df[col_cat]
y = df[col_tar]

### Nümerik DataFrame'deki (varsa) kayıp verileri her bir kolon için o kolonun ortalama değeri ile doldurun. DataFrame'deki her bir kolonu kendi içinde min-max şeklinde normalize edin.

In [99]:
df_num.describe()


,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc
count,391.000000,388.000000,353.000000,354.000000,351.000000,356.000000,381.000000,383.000000,313.000000,312.000000,348.000000,329.000000,294.000000,269.000000
mean,51.483376,76.469072,1.017408,1.016949,0.450142,148.036517,57.425722,3.072454,137.528754,4.627244,12.526437,38.884498,8406.122449,4.707435
std,17.169714,13.683637,0.005717,1.352679,1.099191,79.281714,50.503006,5.741126,10.408752,3.193904,2.912587,8.990105,2944.474190,1.025323
min,2.000000,50.000000,1.005000,0.000000,0.000000,22.000000,1.500000,0.400000,4.500000,2.500000,3.100000,9.000000,2200.000000,2.100000
25%,42.000000,70.000000,1.010000,0.000000,0.000000,99.000000,27.000000,0.900000,135.000000,3.800000,10.300000,32.000000,6500.000000,3.900000
50%,55.000000,80.000000,1.020000,0.000000,0.000000,121.000000,42.000000,1.300000,138.000000,4.400000,12.650000,40.000000,8000.000000,4.800000
75%,64.500000,80.000000,1.020000,2.000000,0.000000,163.000000,66.000000,2.800000,142.000000,4.900000,15.000000,45.000000,9800.000000,5.400000
max,90.000000,180.000000,1.025000,5.000000,5.000000,490.000000,391.000000,76.000000,163.000000,47.000000,17.800000,54.000000,26400.000000,8.000000


In [100]:
df_num["age"].mean()

51.48337595907928

In [101]:
df_num_na = df_num.columns[df_num.isna().sum()>0]
for i in df_num_na:
    a = df[i].mean()
    df_num[i].fillna(a,inplace=True,axis=0)    
df_num

C:\Users\dogan\AppData\Local\Continuum\anaconda2\envs\python37\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc
0,48.0,80.0,1.020,1.0,0.0,121.000000,36.0,1.2,137.528754,4.627244,15.4,44.0,7800.0,5.200000
1,7.0,50.0,1.020,4.0,0.0,148.036517,18.0,0.8,137.528754,4.627244,11.3,38.0,6000.0,4.707435
2,62.0,80.0,1.010,2.0,3.0,423.000000,53.0,1.8,137.528754,4.627244,9.6,31.0,7500.0,4.707435
3,48.0,70.0,1.005,4.0,0.0,117.000000,56.0,3.8,111.000000,2.500000,11.2,32.0,6700.0,3.900000
4,51.0,80.0,1.010,2.0,0.0,106.000000,26.0,1.4,137.528754,4.627244,11.6,35.0,7300.0,4.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,140.000000,49.0,0.5,150.000000,4.900000,15.7,47.0,6700.0,4.900000
396,42.0,70.0,1.025,0.0,0.0,75.000000,31.0,1.2,141.000000,3.500000,16.5,54.0,7800.0,6.200000
397,12.0,80.0,1.020,0.0,0.0,100.000000,26.0,0.6,137.000000,4.400000,15.8,49.0,6600.0,5.400000
398,17.0,60.0,1.025,0.0,0.0,114.000000,50.0,1.0,135.000000,4.900000,14.2,51.0,7200.0,5.900000


In [102]:
for i in df_num.columns:
    a = df_num[i] - df_num[i].min()
    b = df_num[i].max() - df_num[i].min()
    df_num[i] = a/b
df_num

C:\Users\dogan\AppData\Local\Continuum\anaconda2\envs\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc
0,0.522727,0.230769,0.75,0.2,0.0,0.211538,0.088575,0.010582,0.839298,0.047803,0.836735,0.777778,0.231405,0.525424
1,0.056818,0.000000,0.75,0.8,0.0,0.269309,0.042362,0.005291,0.839298,0.047803,0.557823,0.644444,0.157025,0.441938
2,0.681818,0.230769,0.25,0.4,0.6,0.856838,0.132221,0.018519,0.839298,0.047803,0.442177,0.488889,0.219008,0.441938
3,0.522727,0.153846,0.00,0.8,0.0,0.202991,0.139923,0.044974,0.671924,0.000000,0.551020,0.511111,0.185950,0.305085
4,0.556818,0.230769,0.25,0.4,0.0,0.179487,0.062901,0.013228,0.839298,0.047803,0.578231,0.577778,0.210744,0.423729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.602273,0.230769,0.75,0.0,0.0,0.252137,0.121951,0.001323,0.917981,0.053933,0.857143,0.844444,0.185950,0.474576
396,0.454545,0.153846,1.00,0.0,0.0,0.113248,0.075738,0.010582,0.861199,0.022472,0.911565,1.000000,0.231405,0.694915
397,0.113636,0.230769,0.75,0.0,0.0,0.166667,0.062901,0.002646,0.835962,0.042697,0.863946,0.888889,0.181818,0.559322
398,0.170455,0.076923,1.00,0.0,0.0,0.196581,0.124519,0.007937,0.823344,0.053933,0.755102,0.933333,0.206612,0.644068


### Kategorik DataFrame'deki (varsa) kayıp verileri her bir kolon için o kolonda en sık tekrar eden değer ile doldurun.

In [103]:
df_cat_na = df_cat.columns[df_cat.isna().sum()>0]
df_cat_na
for i in df_cat_na:
    a = df_cat[i].value_counts().keys()[0]
    df_cat[i].fillna(a,inplace=True,axis=0)
df_cat

C:\Users\dogan\AppData\Local\Continuum\anaconda2\envs\python37\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane
0,normal,normal,notpresent,notpresent,yes,yes,no,good,no,no
1,normal,normal,notpresent,notpresent,no,no,no,good,no,no
2,normal,normal,notpresent,notpresent,no,yes,no,poor,no,yes
3,normal,abnormal,present,notpresent,yes,no,no,poor,yes,yes
4,normal,normal,notpresent,notpresent,no,no,no,good,no,no
...,...,...,...,...,...,...,...,...,...,...
395,normal,normal,notpresent,notpresent,no,no,no,good,no,no
396,normal,normal,notpresent,notpresent,no,no,no,good,no,no
397,normal,normal,notpresent,notpresent,no,no,no,good,no,no
398,normal,normal,notpresent,notpresent,no,no,no,good,no,no


In [104]:
for i in df_cat.columns:
    print(df_cat[i].unique())

['normal' 'abnormal']
['normal' 'abnormal']
['notpresent' 'present']
['notpresent' 'present']
['yes' 'no']
['yes' 'no' ' yes']
['no' 'yes']
['good' 'poor']
['no' 'yes']
['no' 'yes']


### Kategorik DataFrame'deki kolonlara encoding uygulayın. Burada istediğiniz yöntemi uygulayabilirsiniz. yes/no gibi olan değerleri 1/0 olarak replace edebilir; gerekenlere one-hot encoding uygulayabilirsiniz. Veya hepsine one-hot encoding uygulayabilirsiniz. Size kalmış... Eğer one-hot uyguluyorsanız sonrasında one-hot ile değiştirdiğiniz eski kolonları düşürmeniz/kaldırmanız gerekiyor. Yani hiç text kalmamalı

In [105]:
df_cat_test = df_cat.copy()
for i in df_cat_test.columns:
    df_dummi = pd.get_dummies(df_cat_test[i],prefix=i,drop_first=True)
    df_cat_test.drop(columns=i, axis=1,inplace=True)
    df_cat_test = pd.concat([df_cat_test,df_dummi],axis=1)
df_cat_test

,rbc_normal,pc_normal,pcc_present,ba_present,htn_yes,dm_no,dm_yes,cad_yes,appet_poor,pe_yes,ane_yes
0,1,1,0,0,1,0,1,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,0
2,1,1,0,0,0,0,1,0,1,0,1
3,1,0,1,0,1,1,0,0,1,1,1
4,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
395,1,1,0,0,0,1,0,0,0,0,0
396,1,1,0,0,0,1,0,0,0,0,0
397,1,1,0,0,0,1,0,0,0,0,0
398,1,1,0,0,0,1,0,0,0,0,0


### Nümerik ve Katagorik DataFrameleri axis=1 ile birleştirip feature (X) DataFrame'ini oluşturun.

In [107]:
X = pd.concat([df_num,df_cat_test],axis=1)
print(X)
print(X.isna().sum().sum()) # 0 olmalıdır.

          age        bp    sg   al   su       bgr        bu        sc  \
0    0.522727  0.230769  0.75  0.2  0.0  0.211538  0.088575  0.010582   
1    0.056818  0.000000  0.75  0.8  0.0  0.269309  0.042362  0.005291   
2    0.681818  0.230769  0.25  0.4  0.6  0.856838  0.132221  0.018519   
3    0.522727  0.153846  0.00  0.8  0.0  0.202991  0.139923  0.044974   
4    0.556818  0.230769  0.25  0.4  0.0  0.179487  0.062901  0.013228   
..        ...       ...   ...  ...  ...       ...       ...       ...   
395  0.602273  0.230769  0.75  0.0  0.0  0.252137  0.121951  0.001323   
396  0.454545  0.153846  1.00  0.0  0.0  0.113248  0.075738  0.010582   
397  0.113636  0.230769  0.75  0.0  0.0  0.166667  0.062901  0.002646   
398  0.170455  0.076923  1.00  0.0  0.0  0.196581  0.124519  0.007937   
399  0.636364  0.230769  1.00  0.0  0.0  0.232906  0.042362  0.009259   

          sod       pot  ...  pc_normal  pcc_present  ba_present  htn_yes  \
0    0.839298  0.047803  ...          1       

### y'deki değerleri notckd:0 , ckd:1 olacak şekilde replace edin.

In [113]:
y = y.replace({"notckd":0,"ckd":1})#kod
print(y) #sadece 0 ve 1 olmalıdır.

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

### X ve y'ye test oranı %30 olacak şekilde train-test split uygulayın.
### X_train, X_test, y_train, y_test verilerini oluşturun.

In [114]:
X_train, X_test, y_train, y_test = ms.train_test_split(X,y,test_size=0.3,random_state=42)

### Aşağıda Gaussian Naive Bayes ile predict yapan ve sonuçları çıkartan bir fonksiyon tanımlanmıştır.
### Bu fonksiyonun çıktılarına confusion matrix ekleyiniz. Çıktılar şu şekilde olmalıdır:
`(y_true,y_pred,y_proba,conf_mat)`

In [115]:
#sklearn.naive_bayes.GaussianNB
from sklearn.naive_bayes import GaussianNB

def predict_gnb(X_train,X_test,y_train,y_test):
    
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    
    y_proba = clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    
    y_true = y_test
    
    conf_mat = confusion_matrix(y_test,y_pred)
    return (y_true,y_pred,y_proba,conf_mat)


### Yukarıda modifiye ettiğiniz fonksiyonun aynısını(confusion matrix ile birlikte) Decision Tree için hazırlayın:

In [116]:
#sklearn.tree.DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

def predict_dt(X_train,X_test,y_train,y_test):
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)
    
    y_proba = clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    
    y_true = y_test
    
    conf_mat = confusion_matrix(y_test,y_pred)
    return (y_true,y_pred,y_proba,conf_mat)


### Yukarıda modifiye ettiğiniz fonksiyonun aynısını(confusion matrix ile birlikte) KNN için hazırlayın:

In [120]:
#sklearn.neighbors.KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

def predict_knn(X_train,X_test,y_train,y_test,k=5):
    #kod
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train,y_train)
    
    y_proba = clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    
    y_true = y_test
    
    conf_mat = confusion_matrix(y_test,y_pred)
    return (y_true,y_pred,y_proba,conf_mat)

### Yukarıda modifiye ettiğiniz fonksiyonun aynısını(confusion matrix ile birlikte) SVM için hazırlayın:
*sklearn.svm.SVC varsayılan ayarlarında class probabilities'i çıkartmıyor (clf.predict_proba çalışmıyor.)*

*Bunu etkinleştirmek için tanımlarken attributeları arasında probability'yi True yapmak gerekiyor.*

In [1]:
#sklearn.svm.SVC
from sklearn.svm import SVC

def predict_svm(X_train,X_test,y_train,y_test,kernel='linear',prob=True)
    clf = SVC(kernel=kernel,probability=prob)
    #kod
    
    if prob:
        y_proba = clf.predict_proba(X_test)
        return (y_true,y_pred,y_proba,conf_mat)
    else:
        return (y_true,y_pred,conf_mat)

### Ödev1'deki confusion matrix'den performans metriklerini çıkartan fonksiyonunuzu yapıştırın.

### Her bir classifier için confusion matrixleri oluşturarak performance_metrics fonksiyonuyla sonuçları yazdırın.
### KNN için K = 1,3,5,7 değerlerini ayrı ayrı deneyin.
### SVM için ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ kernellerini ayrı ayrı deneyin.
#### * Her bir classifier için sonuçları ekrana yazdırmadan önce classifier'ın adını da yazdırın.

### Çıkan sonuçlara göre en iyi sonuç veren classifier'ı ve özelliğini belirleyin (KNN ise k, SVM ise kernel gibi)
### Bu adımdan sonra bu classifier'ı bu özelliklerle kullanacaksınız.

### Bu adımda X DataFrame'inize(features) PCA dönüşümü uygulayın ve PCA_X DataFrame'ini elde edin.

In [ ]:
x_pca = #transformation
PCA_X = pd.DataFrame(x_pca)

### PCA dönüşümünde hangi kolonun ne kadar bilgi taşıdığını yazdırın. (explained variance ratio)

### Bilginin en az %90'ını kapsayacak şekilde ilk birkaç kolonu seçin ve X_necessary DataFrame'ini oluşturun.

### n=10 olacak şekilde X_necessary ve y'ye n-Fold Cross Validation (shuffle ile birlikte) uygulayarak
### X_trains, X_tests, y_trains, y_tests listelerini oluşturun.

### Bu adımda şu iterasyonu yaparak X_trains, X_tests, y_trains, y_tests listelerindeki her bir eleman için
### performans metriklerini hesaplamanız gerekiyor: (Kod fikir vermesi için)

n fold sayısı olmak üzere:
```python
accuricies = []

for i in range(n):
    X_train = X_trains[i]
    X_test = X_tests[i]
    y_train = y_trains[i]
    y_test = y_tests[i]
    
    predict_classifier(X_train,X_test,y_train,y_test) #yukarıda seçtiğiniz en iyi performans veren classifier
    accuricies.append(accuracy) #accuracy

mean_acc = mean(accuracies)
print(mean_acc)
```

1. En iyi performans veren classifier'ı seçerken neye göre seçtiniz? Neden?

2. En son adımda ortalama accuracy'yi hesapladık. Cross validation'daki setler için başka neleri hesaplamamız mantıklı olurdu?

3. Öğrendikleriniz / Katkılarınız

4. (Varsa) eklemek istedikleriniz

Grup Üyesi 1 İsim : 

1. 

2. 

3. 

4. 


---
Grup Üyesi 2 İsim : 

1. 

2. 

3. 

4. 


---
Grup Üyesi 3 İsim : 

1. 

2. 

3. 

4. 


---
Grup Üyesi 4 İsim : 

1. 

2. 

3. 

4. 


---
Grup Üyesi 5 İsim : 

1. 

2. 

3. 

4. 


---
Grup Üyesi 6 İsim : 

1. 

2. 

3. 

4. 


---
Grup Üyesi 7 İsim : 

1. 

2. 

3. 

4. 

